# Calculating $e$ with parallel code

Recall in the `1-Nonparallel` notebook, we found a way to calculate $e$ with a sum...

$$e = \sum^\infty_{i=0} \frac{1}{i!}$$

We wrote some julia code to do this...

```julia
calculateTerm(i) = 1.0/factorial(i)
calculate_e(nTerms) = [ calculateTerm(i) for i in 0:nTerms ] |> sum
```

We executed this code in a non-parallel program, that is $e$ was calculated serially in on process by one processor.

With MPI, we can have terms calculated simultaneously with parallel processes. The simplest way to do this is to have each rank calculate a term and then add them all up. So the number of terms in the sum will be the same as the number of ranks. An interesting challenge with this code is how to do the sum? Each rank will only know the value of its term. We will need to use MPI communication in order to have each rank communicate its term to a *Root rank*. The root rank is a special rank that will collect information from the other ranks and process it - in this case, the root rank will sum up the terms. MPI doesn't care which rank is the root rank, but it is customary to choose Rank 0 as the root. 

In [1]:
# This cell should add the packages you need for 
# this notebook (and other notebooks in this directory)
# You should get a message saying 
# "Activating project at `.../TryMPI/notebooks` where the ... is the 
# absolute path to the directory of this repository and these notebooks.
using Pkg
Pkg.activate(".")
Pkg.instantiate()

# This cell will appear in other notebooks as well

  Activating project at `~/Development/HighVelocityJuliaAnalysis/TryMPI/notebooks`


There are actually two ways we can do the sum. 1) We can use the `MPI.Gather` function to pass an array to the Root rank and have that rank do the sum. Or 2) we can use `MPI.Reduce` to have MPI calculate the sum for us. Let's look at both ways. 

See https://mpitutorial.com/tutorials/mpi-scatter-gather-and-allgather/ for a good tutorial on `MPI.Gather` (but the code is C instead of Julia). 

### Gather and sum

Look at `calc_e_gather.jl` . You can run it from the terminal with

```bash
~/.julia/bin/mpiexecjl --project -n 4 julia calc_e_gather.jl 
```

There are some new lines compared to what you saw in `displayRank.jl`.

```julia
rootRank = 0
iAmRoot = myRank == rootRank
```

Here we are saying that the root rank is Rank 0. We could have picked any rank, but 0 is a good choice because even an MPI program with only one rank will always have a Rank 0. The `iAmRoot` line is just an easy short hand to determine if this rank is the root rank. Remember that MPI runs copies of this code simultaneously. Only one copy will be Rank 0 and thus only that copy will have `iAmRoot` as `true`. The other ranks will have `iAmRoot` as `false`. 

```julia
# Function for calculating a term
calculateTerm(i) = 1.0/factorial(i)

# Calculate my term
myTerm = calculateTerm(myRank)
```

Here we define the function to calculate a term of $e$ and then we use it. Note that we're using the rank as $i$ in the function. This is convenient, as rank is an integer that starts with zero. 

```julia
# Gather terms to the Root rank
terms = MPI.Gather(myTerm, rootRank, comm)

print("I am rank $(myRank) and terms are $(terms)\n")
```

This is where the action happens. We give `MPI.Gather` our value, which Rank is root (in this case 0), and the communicator. The value of `terms` depends on the rank.
- The Root Rank (rank 0) will get an array. The ith entry will be the term from the (i-1)th rank. That is entry 1 in the array is the term from rank 0. Entry 2 in the array is the term from rank 1 (here is where Julia's 1-based array indexing gets annoying).
- The other ranks will have `term` set to `nothing`. `nothing` is a special Julia value that means what it says - it is nothing. You can't do any operations on it, except to compare (e.g. `term == nothing` would be true).

The `print` line verifies the above. 

```julia
if iAmRoot
    e_estimate = sum(terms)
    print("Estimate of e with $(nRanks) terms is $(e_estimate)\n")
end
```

This last part only runs in the root rank (the other ranks will be idle). It calculates the sum and prints it, giving our estimate of $e$. Can you think of why this code must only run in the root rank (e.g. why the `if` statement)?


### Reduce

Look at `calc_e_reduce.jl` and try to run it. It looks very similar to the program we just examined, `calc_e_gather.jl`, but with a big difference. Instead of gathering all of the terms to the Root Rank and having that rank calculate the sum, MPI actually calculates the sum for us with the `MPI.Reduce` function call. 

```julia
e_estimate = MPI.Reduce(myTerm, +, rootRank, comm)

print("I am rank $(myRank) and reduce returned $(e_estimate)\n")
```

For `MPI.Reduce`, each rank passes in a value (`myTerm` in this case) and MPI *reduces* those values by running the specified operation (`+` in this case). The result of those operations is one number, and that is returned to the Root Rank. The other ranks get the Julia `nothing` value (just like with `MPI.Gather`).

```julia
if iAmRoot
    print("The estimate for e is $(e_estimate)")
end
```

Here, we print the result nicely. Why do we need the `if` statement?

You may be wondering how we can use `+` as a function in the `MPI.Reduce` call. `+` is actually a Julia function!

In [1]:
+(2,2)

4

`+` has a special *infix* form where `a + b` gets interpreted as `+(a,b)`.

There are some advantages of using `MPI.Reduce` over `MPI.Gather`. With `MPI.Gather`, we returned an array of terms to the Root rank, and then that rank did the sum. That sum happened serially. If we had an extremely large list of terms, then that sum could have taken a long time to compute. Having `MPI.Reduce` do the sum means that the sum happens in parallel on many processors and this can be much more efficient than having one rank do the sum itself. The MPI reduce [tutorial](https://mpitutorial.com/tutorials/mpi-reduce-and-allreduce/) explains this pretty well. 